# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233915225577                   -0.50    7.0         
  2   -7.249807035680       -1.80       -1.38    1.0   9.42ms
  3   -7.250788599725       -3.01       -1.76    2.0   10.6ms
  4   -7.250950254786       -3.79       -1.86    1.0   9.31ms
  5   -7.251331126924       -3.42       -2.70    1.0   9.27ms
  6   -7.251337554349       -5.19       -3.09    1.0   9.78ms
  7   -7.251338677125       -5.95       -3.52    2.0   10.8ms
  8   -7.251338792640       -6.94       -4.12    1.0   9.48ms
  9   -7.251338798110       -8.26       -4.69    2.0   11.6ms
 10   -7.251338798646       -9.27       -5.17    2.0   10.9ms
 11   -7.251338798691      -10.35       -5.49    2.0   12.0ms
 12   -7.251338798703      -10.91       -6.09    1.0   9.84ms
 13   -7.251338798704      -11.93       -6.51    3.0   12.6ms
 14   -7.251338798705      -13.05       -6.90    1.0   10.0ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06352735415096974
[ Info: Arnoldi iteration step 2: normres = 0.4191755508724177
[ Info: Arnoldi iteration step 3: normres = 1.023033584965217
[ Info: Arnoldi iteration step 4: normres = 0.3018182171693712
[ Info: Arnoldi iteration step 5: normres = 0.6226945052562471
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (6.64e-02, 9.00e-02, 5.65e-01, 2.35e-01, 2.97e-02)
[ Info: Arnoldi iteration step 6: normres = 0.22831363666255963
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.18e-02, 1.04e-01, 1.43e-01, 1.13e-01, 2.86e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09143467588328275
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.16e-04, 8.96e-03, 1.11e-02, 5.59e-02, 6.56e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09341794649896505
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.07e-0